<a href="https://colab.research.google.com/github/shengyangz/colab/blob/main/svc_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ls

sample_data/


In [1]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
from joblib import Parallel, delayed
import tqdm
import jieba
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, recall_score

In [2]:
# prompt: read an excel file

import pandas as pd
df = pd.read_excel('eee.xlsx')


In [3]:
df

,type,text,label
0,停水投诉=>未知原因停水,用户反映此处未知原因停水，已检查表前阀门属打开状态，请我司工作人员核实并立即处理。,y
1,停水投诉=>其它,私自泄露诉求人信息的，视情节严重程度给予责任追究，请注意保密。如不属我司范围必须在一个工作日...,y
2,停水投诉=>未知原因停水,用户反映未知原因停水，已检查表前阀门属打开状态，请立即处理。,y
3,停水投诉=>未知原因停水,用户反映未知原因停水，已检查表前阀门属打开状态，请尽快联系用户处理。（用户未提供联系人姓名信息）,y
4,停水投诉=>同一片区多次停水,私自泄露诉求人信息的，视情节严重程度给予责任追究，请注意保密。（用户未留下联系方式，不需联系...,y
...,...,...,...
514,停水投诉=>同一片区多次停水,用户反映小区水管改造后整栋楼经常出现停水，用户情绪很激动，并认为我司安排的施工方技术不合格，...,y
515,业务受理=>单位用户=>报停用水预受理,单位用户办理报停用水，请核实收取以下资料：经办人（周芳娟）身份证、《报停、恢复用水申请表》（...,n
516,业务受理=>单位用户=>报停用水预受理,单位用户办理报停用水，请核实收取以下资料：经办人（周芳娟）身份证、《报停、恢复用水申请表》（...,n
517,业务受理=>个人用户=>恢复用水预受理,个人用户办理恢复用水，请核实收取以下资料： 李先生的身份证（验原件收复印件）、用水地址的合法...,n


In [4]:
def jieba_cut(x, sep=' '):
    return sep.join(jieba.cut(x, cut_all=False))

In [5]:
%%time
df['text_cut'] = Parallel(n_jobs=4)(
    delayed(jieba_cut)(x) for x in tqdm.tqdm_notebook(df['text'].values)
)

<timed exec>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/519 [00:00<?, ?it/s]

CPU times: user 418 ms, sys: 111 ms, total: 529 ms
Wall time: 6.63 s


In [6]:
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)

In [7]:
print(f"train: {train_df.shape}, test: {test_df.shape}")

train: (415, 4), test: (104, 4)


In [8]:
train_df.head()

,type,text,label,text_cut
433,停水投诉=>计划停水未提前通知（含延迟通水）,用户来电咨询坪山区东纵路计划停水延时通水未及时通知，已向用户解释，要求跟进处理。（用户未提供...,y,用户 来电 咨询 坪 山区 东纵路 计划 停水 延时 通水 未 及时 通知 ， 已 向 用户...
517,业务受理=>个人用户=>恢复用水预受理,个人用户办理恢复用水，请核实收取以下资料： 李先生的身份证（验原件收复印件）、用水地址的合法...,n,个人用户 办理 恢复 用水 ， 请 核实 收取 以下 资料 ： 李先生 的 身份证 （ ...
208,查询/咨询=>其它咨询,私自泄露诉求人信息的，视情节严重程度给予责任追究，请注意保密。如不属我司范围必须在一个工作日...,y,私自 泄露 诉求 人 信息 的 ， 视 情节严重 程度 给予 责任 追究 ， 请 注意 保密...
332,停水投诉=>未知原因停水,用户反映未知原因停水，已检查表前阀门属打开状态，请立即处理。,y,用户 反映 未知 原因 停水 ， 已 检查表 前 阀门 属 打开 状态 ， 请 立即 处理 。
220,查询/咨询=>其它咨询,私自泄露诉求人信息的，视情节严重程度给予责任追究，请注意保密。如不属我司范围必须在一个工作日...,y,私自 泄露 诉求 人 信息 的 ， 视 情节严重 程度 给予 责任 追究 ， 请 注意 保密...


In [15]:
def count_vect_feature(feature, df, max_features=5000):
    start_time = time.time()
    cv = CountVectorizer(max_features=max_features,
                             ngram_range=(1, 1),
                             stop_words='english')
    X_feature = cv.fit_transform(df[feature])
    print('Count Vectorizer `{}` completed in {} sec.'.format(feature, round(time.time() - start_time,2)))
    return X_feature, cv

In [16]:
X_text, cv = count_vect_feature('text_cut', train_df, 50000)

Count Vectorizer `text_cut` completed in 0.05 sec.


In [17]:
target =  'label'
X = X_text
y = train_df[target].values
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [18]:
train_X.shape, valid_X.shape, train_y.shape, valid_y.shape

((332, 1777), (83, 1777), (332,), (83,))

In [19]:
%%time
clf_svc = SVC(kernel='linear')
clf_svc = clf_svc.fit(train_X, train_y)

CPU times: user 18.3 ms, sys: 0 ns, total: 18.3 ms
Wall time: 24.2 ms


In [21]:
predicted_valid = clf_svc.predict(valid_X)
prediction_acc = np.mean(predicted_valid == valid_y)
prediction_f1_score = f1_score(valid_y, predicted_valid, average='weighted')
prediction_recall = recall_score(valid_y, predicted_valid, average='weighted')
print("Valid:\n========================================================")
print(f"Feature: {target} \t| Prediction accuracy: {prediction_acc}")
print(f"Feature: {target} \t| Prediction F1-score: {prediction_f1_score}")
print(f"Feature: {target} \t| Prediction recall: {prediction_recall}")

Valid:
Feature: label 	| Prediction accuracy: 0.9759036144578314
Feature: label 	| Prediction F1-score: 0.9640023508668822
Feature: label 	| Prediction recall: 0.9759036144578314
